# Coronavirus data prep & modelling

 Outstanding tasks:
 - refactor code for iloc issue with aggregations
 - check country aggregations

## Libraries and data 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from datetime import timedelta
from dateutil import parser
from scipy.integrate import odeint
import plotly.express as px
from sklearn.linear_model import LinearRegression

In [ ]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [ ]:
df_cases = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
df_deaths = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
df_recovered = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
world_pop = pd.read_csv("data_feeds/population_world_data.csv", encoding='latin-1')
hosp_beds = pd.read_csv("data_feeds/hospital_beds.csv").drop(['country'], axis=1)
oxford = pd.read_csv("data_feeds/OxCGRT_Download_latest_data.csv",parse_dates = ['Date'], encoding="ISO-8859-1")

## Transposed and merge data

In [ ]:
# Process and merge datasets

latest_date = df_cases.columns[-1]
oxford['country_map'] = oxford['CountryName'].apply(lambda x: 'Korea, South' if x == 'South Korea' else 'US' if x == 'United States' else x)
oxford = oxford[['CountryName', 'country_map','Date','StringencyIndex']]
world_pop['country_map'] = world_pop['Country'].apply(lambda x: 'Korea, South' if x == 'South Korea' else 'US' if x == 'United States' else x)
df_merge = df_cases.merge(world_pop[['country_map', 'Population_2020']], how='left', left_on='Country/Region', right_on='country_map')
df_merge = df_merge.merge(hosp_beds, how='left', left_on='Country/Region', right_on='country_map')
df_merge['hospital_beds'] = df_merge['Population_2020'] * df_merge['hospital_beds_per_10000'] / 10000

In [ ]:
def df_transpose(df):
    df2 = df.drop(['Province/State','Lat','Long'],axis=1).groupby(['Country/Region']).sum()
    df3 = df2.reset_index().T
    df3.columns = df3.iloc[0]
    df3.drop(df3.index[0], inplace=True)
    df3.index.rename('date', inplace=True)
    df3.reset_index(inplace=True)    
    df3['date'] = df3.date.apply(lambda x: parser.parse(x))
    df3.set_index('date', inplace=True)
    df3 = df3.apply(pd.to_numeric)
    df3.reset_index(inplace=True)
    return df3

In [ ]:
def df_melting(df, var):
    df2 = pd.melt(df, id_vars=['date'], value_vars = df.columns.drop('date'),var_name='country', value_name=var)
    return df2

In [ ]:
# Transpose all feeds
df_t_cases = df_transpose(df_cases)
df_t_deaths = df_transpose(df_deaths)
df_t_recovered = df_transpose(df_recovered)

# Combine country columns to index
df_t_cases = df_melting(df_t_cases, 'actual_cases')
df_t_deaths = df_melting(df_t_deaths, 'actual_deaths')
df_t_recovered = df_melting(df_t_recovered, 'actual_recovered')

# Merge datasets
df_comb = df_t_cases.merge(df_t_deaths, on = ['date', 'country'], how='left')
df_comb = df_comb.merge(df_t_recovered, on = ['date', 'country'], how='left')

In [ ]:
# Data processing

df_comb = df_comb.merge(world_pop[['country_map', 'Population_2020']], how='left', left_on='country', right_on='country_map')
df_comb = df_comb.merge(hosp_beds, how='left', on='country_map')
#df_comb = df_comb.merge(oxford, how='left', left_on =['country_map','date'], right_on=['CountryName', 'Date'])
df_comb = df_comb.merge(oxford, how='left', left_on =['country','date'], right_on=['country_map', 'Date'])
df_comb['hospital_beds'] = df_comb['Population_2020'] * df_comb['hospital_beds_per_10000'] / 10000
df_comb['fatality_rate'] = (df_comb['actual_deaths']/df_comb['actual_cases']).fillna(0)

## SIR model

In [ ]:
# Set parameters, based on https://drive.google.com/file/d/1DqfSnlaW6N3GBc5YKyBOCGPfdqOsqk1G/view

gamma = 1./10          # Mean period while contagious
hosp_rate = 0.05       # 5% hospitalisation rate
icu_rate = 0.025       # 2.5% ICU rate
fr = 0.01/10           # Per day fatality rate
test_multiplier = 10   # Assumed multiplier for unconfirmed cases
start_date_dt = date(month = 3,day = 1,year = 2020)
time_window = 180

In [ ]:
# Define simple SIR model

t = np.linspace(0, time_window, time_window)
def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I #- fr * I 
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

In [ ]:
# Run projectionloop

projections = pd.DataFrame()

for ctry in df_cases[['Country/Region', latest_date]].sort_values(by=latest_date, ascending=False).head(30)['Country/Region'].unique():
    for beta in np.linspace(0.05,0.30,11):
        for max_pop in [0.2,0.5,0.8]:

            t = np.linspace(0, time_window, time_window+1)

            I0 = test_multiplier * df_comb[(df_comb['country'] == ctry) & (df_comb['date'] == start_date_dt)].actual_cases.values[0]
            R0 = df_comb[(df_comb['country'] == ctry) & (df_comb['date'] == start_date_dt)].actual_recovered.values[0]
            N = max_pop * df_merge[df_merge['Country/Region'] == ctry]['Population_2020'].iloc[0]
            S0 = N - I0 - R0
            y0 = S0, I0, R0
            ret = odeint(deriv, y0, t, args=(N, beta, gamma))
            S, I, R = ret.T

            H = I * hosp_rate
            ICU = I * icu_rate
            D = I * fr
            fatalities = D.cumsum()
            beds = np.repeat(df_merge[df_merge['Country/Region'] == ctry]['hospital_beds'].iloc[0], time_window+1)

            # Output results
            new = pd.DataFrame(data=np.array([t,S,I,R,H,ICU,D,fatalities,beds]).T, 
                                   columns=['days','projected_susceptible','projected_infections','projected_recovered','projected_hospitalisation',
                                           'projected_icu','projected_deaths','projected_fatalities','projected_beds'])
            new['country'] = ctry
            new['max_pop'] = max_pop
            new['beta'] = beta
            projections = pd.concat([projections,new], axis=0)

In [ ]:
# Filter for X countries with most cases

ctry_list = df_cases[['Country/Region', latest_date]].sort_values(by=latest_date, ascending=False).head(50)['Country/Region'].unique()
projections['date'] = pd.to_datetime(projections.days.apply(lambda x: start_date_dt + timedelta(x)))
df_final = projections.merge(df_comb, how='left', on=['country','date'])
print(df_final.shape)
df_final[df_final['country'].isin(ctry_list)]
print(df_final.shape)

In [ ]:
# Add new fields

df_final['new_cases'] = (df_final['actual_cases'] - df_final['actual_cases'].shift(periods=1))#.fillna(0)
df_final['new_deaths'] = (df_final['actual_deaths'] - df_final['actual_deaths'].shift(periods=1))#.fillna(0)
#df_final = df_final.merge(oxford, how='left', left_on =['country','date'], right_on=['country_map', 'Date'])

## Logistic regression

In [ ]:
# Panel regression - take logs of case numbers and dummify country

df_final['projections_flag'] = df_final['actual_cases'].isna()
df_final['ln_actual_cases'] = np.log(1+ df_final['actual_cases'])
df_cut = df_final[(df_final.beta == 0.2) & (df_final.max_pop == 0.5)][['days','country','ln_actual_cases']]
countries = pd.get_dummies(df_cut['country']).iloc[:, :-1]
df = pd.concat([df_cut, countries], axis=1).drop(['country'], axis=1)

In [ ]:
# Split train and test sets

df_train = df[df['days'] <= 24 ]
df_test = df[(df['days'] > 24) & (df['days'] <= 25)]

y_train = df_train['ln_actual_cases']
y_test = df_test['ln_actual_cases']

df_train.drop('ln_actual_cases', axis=1, inplace=True)
df_test.drop('ln_actual_cases', axis=1, inplace=True)

print(df_train.shape, df_test.shape)

In [ ]:
# Fit regression model

lr = LinearRegression(n_jobs=-1,fit_intercept = True)
lr.fit(df_train, y_train)
print("train score: " + str(lr.score(df_train, y_train)))
print("test score: " + str(lr.score(df_test, y_test)))

In [ ]:
# Output model coefficients

for i in range(0,len(df_train.columns)):
    print(df_train.columns[i] + " : " + str(round(lr.coef_[i],4)))

In [ ]:
df['ln_log_reg_preds'] = lr.predict(df.drop(['ln_actual_cases'], axis=1))
df['log_reg_preds'] = np.exp(df['ln_log_reg_preds'])-1
df_merge = df

In [ ]:
def get_country(row):
    for c in df_merge.iloc[:,2:-2].columns:
        if row[c]==1:
            return c

In [ ]:
df_merge['country'] = df_merge.iloc[:,1:-2].apply(get_country, axis=1)
df_merge['country'].fillna('United Kingdom', inplace=True)
df_final_v2 = df_final.merge(df_merge[['days','country','log_reg_preds']], how='left', on=['days', 'country'])
df_final_v2['StringencyIndex'].fillna(method='ffill', inplace=True)
print(df_final_v2.shape)

## Generate output

In [ ]:
df_final.to_csv("dash/df_final.csv", index=False)
df_final_v2.to_csv("dash/df_final_v2.csv", index=False)

## Visuals - projections

In [ ]:
df_filt = df_final_v2[df_final_v2.days < 30][['date','country','max_pop','beta','actual_cases','projected_infections','log_reg_preds']].melt(id_vars = ['date', 'country','max_pop','beta'], var_name = 'projection')
px.line(df_filt[(df_filt.country == 'United Kingdom') & (df_filt.max_pop == 0.5) & (df_filt.beta == 0.2)],
        x='date', y='value', color='projection' )

In [ ]:
df_filt = df_final_v2[df_final_v2.days < 30][['date','country','max_pop','beta', 'StringencyIndex']].melt(id_vars = ['date', 'country','max_pop','beta'], var_name = 'projection')
px.line(df_filt[(df_filt.country == 'United Kingdom') & (df_filt.max_pop == 0.5) & (df_filt.beta == 0.2)],
        x='date', y='value', color='projection' )

## Visuals - actuals

In [ ]:
df_comb['cases_above_0'] =  np.where(df_comb['actual_cases']>0, True, False)
df_comb['days_since_first_case'] = df_comb.groupby(by=['country'])['cases_above_0'].transform(lambda x: x.cumsum())
df_comb['deaths_above_0'] =  np.where(df_comb['actual_cases']>0, True, False)
df_comb['days_since_first_death'] = df_comb.groupby(by=['country'])['deaths_above_0'].transform(lambda x: x.cumsum())
ctry_list = df_cases[['Country/Region', latest_date]].sort_values(by=latest_date, ascending=False).head(15)['Country/Region'].unique()

In [ ]:
df_filt = df_comb[df_comb.country.isin(ctry_list)][['date','country','actual_cases']].melt(id_vars = ['date', 'country'],var_name = 'projection')
px.line(df_filt, x='date', y='value', color='country', title='Confirmed cases')

In [ ]:
df_filt = df_comb[df_comb.country.isin(ctry_list)][['date','country','actual_deaths']].melt(id_vars = ['date', 'country'],var_name = 'projection')
px.line(df_filt, x='date', y='value', color='country', title='Confirmed deaths')

In [ ]:
df_filt = df_comb[df_comb.country.isin(ctry_list)][['days_since_first_case','country','actual_deaths']].melt(id_vars = ['days_since_first_case', 'country'],var_name = 'projection')
px.line(df_filt, x='days_since_first_case', y='value', color='country', title='Confirmed cases since first record')

In [ ]:
df_filt = df_comb[df_comb.country.isin(ctry_list)][['days_since_first_death','country','actual_deaths']].melt(id_vars = ['days_since_first_death', 'country'],var_name = 'projection')
px.line(df_filt, x='days_since_first_death', y='value', color='country', title='Confirmed deaths since first record')

In [ ]:
df_filt = df_comb[df_comb.country.isin(ctry_list)][['days_since_first_case','country','StringencyIndex']].melt(id_vars = ['days_since_first_case', 'country'],var_name = 'projection')
px.line(df_filt, x='days_since_first_case', y='value', color='country', title='Policy response since first case')

In [ ]:
df_filt = df_comb[df_comb.country.isin(ctry_list)][['days_since_first_death','country','StringencyIndex']].melt(id_vars = ['days_since_first_death', 'country'],var_name = 'projection')
plt = px.line(df_filt, x='days_since_first_death', y='value', color='country', title='Policy response since first death')
plt.show()

In [ ]:
#plt.savefig('images/policy_response.png')

In [ ]:
plt.write_image('images/policy_response.png')